In [1]:
import sklearn
import numpy as np
import cv2
import os
from sklearn import preprocessing
from sklearn import svm
from sklearn.model_selection import GridSearchCV

In [2]:
def sliding_window(image, step, window_size):
    '''
    滑动窗口,首先要做的是用给定尺寸的滑动图片截取物体识别的图像
    '''
    for y in range(0, image.shape[0]-window_size[0], step):
        for x in range(0, image.shape[1]-window_size[1], step):
            yield (x,y,image[y:y + window_size[0], x:x + window_size[1]])

In [21]:
template=cv2.imread("./image/template.JPG")
template_2=template.reshape(-1)

In [3]:
#遍历训练集，生成训练集数据。
img_list=[]#训练集列表
train_y=[]#训练集标签
for filename in os.listdir("./image/train/0"):
    file="./image/train/0/"+filename
    img_rgb=cv2.imread(file)
    img_array=list(img_rgb.reshape(-1))
    img_list.append(img_array)
    train_y.append(0)
for filename in os.listdir("./image/train/1"):
    file="./image/train/1/"+filename
    img_rgb=cv2.imread(file)
    img_array=list(img_rgb.reshape(-1))
    img_list.append(img_array)
    train_y.append(1)

In [6]:
#转换成np.ndarray类型
train_X = np.array(img_list)

归一化预处理，使用sklearn.preprocessing.StandardScaler类，使用该类的好处在于可以保存训练集中的参数（均值、方差）直接使用其对象转换测试集数据。

In [7]:
#归一化预处理，使用sklearn.preprocessing.StandardScaler类，使用该类的好处在于可以保存训练集中的参数（均值、方差）直接使用其对象转换测试集数据。
train_X_scaler= preprocessing.StandardScaler().fit(train_X)

C:\Anaconda\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [8]:
#归一化训练集
train_X_sc=train_X_scaler.transform(train_X)

C:\Anaconda\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [10]:
#由于训练集的特征数为2000，训练集为1000以下，故采用SVM算法进行回归学习 
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }#使用多少特征点
clf = GridSearchCV(svm.SVC(kernel='rbf', class_weight='balanced'), param_grid)

In [11]:
clf = clf.fit(list(train_X_sc), train_y)

C:\Anaconda\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [13]:
#遍历测试集，生成测试集数据。
img_test=[]
test_y=[]
for filename in os.listdir("./image/test/0"):
    file="./image/test/0/"+filename
    img_rgb=cv2.imread(file)
    img_array=list(img_rgb.reshape(-1))
    img_test.append(img_array)
    test_y.append(0)
for filename in os.listdir("./image/test/1"):
    file="./image/test/1/"+filename
    img_rgb=cv2.imread(file)
    img_array=list(img_rgb.reshape(-1))
    img_test.append(img_array)
    test_y.append(1)

In [14]:
test_X = np.array(img_test)

In [15]:
#归一化测试集数据
test_X_sc=train_X_scaler.transform(test_X)

C:\Anaconda\anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [16]:
#模型训练后，预测测试集
y_pred = clf.predict(test_X_sc)

In [17]:
#预测测试统计
from sklearn.metrics import classification_report
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.87      0.93       139
           1       0.22      1.00      0.36         5

   micro avg       0.88      0.88      0.88       144
   macro avg       0.61      0.94      0.64       144
weighted avg       0.97      0.88      0.91       144



In [18]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

In [19]:
#对源图像进行目标定位
img_rgb_predict = cv2.imread("./image/source.JPG")

In [20]:
template = cv2.imread('./image/object.JPG')

In [21]:
#滑窗法对源图像的截取，并使用模型进行预测，预测为目标对象后，保存坐标
point_list=[]
for image_tuple in sliding_window(img_rgb_predict,2,template.shape):
    img=image_tuple[2]
    img_array=[list(img.reshape(-1))]
    img_X_sc=train_X_scaler.transform(img_array)
    y_pred = clf.predict(img_X_sc)
    if (y_pred==1):
        point = (image_tuple[0], image_tuple[1])
        point_list.append(point)

In [120]:
# 模板匹配后符合要求的所有图案数量
length = len(point_list)
# 设定相同点的阈值
equal_point_threshold = 10
# 如果两个点距离在阈值范围内，则等同，然后用集合去重
for i in range(length):
    for j in range(length):
        if abs(point_list[i][0] - point_list[j][0]) <= equal_point_threshold and abs(
                    point_list[i][1] - point_list[j][1]) <= equal_point_threshold:
            point_list[i] = point_list[j]
point_set = set(point_list)

In [31]:
h, w = template.shape[:2]
for pt in point_set:
    right_bottom = (pt[0] + w, pt[1] + h)
    cv2.rectangle(img_rgb_predict, (pt[0], pt[1]), right_bottom, (0, 0, 255), 1)
cv2.imshow('Detected', img_rgb_predict)
cv2.waitKey(0)

-1

![title](./result.JPG)